In [1]:
#to do: include markdown cells, write comments to explain why things are being dropped
import pandas as pd
import numpy as np

In [1]:
df1988 = pd.read_csv("1988.csv")

In [ ]:
df1998 = pd.read_csv("1998.csv")

In [ ]:
#descriptions of fields available in the document
df1998.head()

In [ ]:
print(len(df1988.index))
print(df1988.columns)
print(len(df1998.index))
print(df1998.columns)

In [6]:
#fields to drop: year, flight number, cancellationcode, delay times, because all values are NaN
#and year is already assumed
df1988 = df1988.drop(['Year','CancellationCode','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','TaxiIn','TaxiOut','FlightNum','TailNum'], axis=1)
df1998 = df1998.drop(['Year','CancellationCode','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay','FlightNum','TailNum'], axis=1)

In [7]:
df1998.head()

In [8]:
def time_to_interval(n):
    if (n<300):
        return '00:00 to 03:00'
    elif (n<600):
        return '03:00 to 06:00'
    elif (n<900):
        return '06:00 to 09:00'
    elif (n<1200):
        return '09:00 to 12:00'
    elif (n<1500):
        return '12:00 to 15:00'
    elif (n<1800):
        return '15:00 to 18:00'
    elif (n<2100):
        return '18:00 to 21:00'
    else:
        return '21:00 to 00:00'

In [9]:
df1998['DepTimeInterval'] = df1998['CRSDepTime'].apply(time_to_interval)
df1988['DepTimeInterval'] = df1988['CRSDepTime'].apply(time_to_interval)
df1998['ArrTimeInterval'] = df1998['CRSArrTime'].apply(time_to_interval)
df1988['ArrTimeInterval'] = df1988['CRSArrTime'].apply(time_to_interval)

In [10]:
def check_weekend(n):
    if (n<=5):
        return 0
    else:
        return 1
df1998['Weekend'] = df1998['DayOfWeek'].apply(check_weekend)
df1988['Weekend'] = df1988['DayOfWeek'].apply(check_weekend)

In [11]:
def check_holiday_1988(n):
    if ((n==101) or
        (n==118) or
        (n==215) or
        (n==430) or
        (n==703) or
        (n==905) or
        (n==1010) or
        (n==1111) or
        (n==1124) or
        (n==1225) or
        (n==1226)):
        return 1
    else:
        return 0    
    
def check_holiday_1998(n):
    if ((n==101) or
        (n==119) or
        (n==216) or
        (n==425) or
        (n==703) or
        (n==704) or
        (n==907) or
        (n==1012) or
        (n==1111) or
        (n==1126) or
        (n==1225) or
        (n==1226)):
        return 1
    else:
        return 0

In [12]:
#i think this is the fastest way to do this in terms of runtime
#need to split this because it keeps killing the kernel
df1988['Holiday'] = ((df1988['Month']*100)+df1988['DayofMonth']).apply(check_holiday_1988)

In [13]:
df1998['Holiday'] = ((df1998['Month']*100)+df1998['DayofMonth']).apply(check_holiday_1998)

In [14]:
d_quantiles_1988 = []
d_quantiles_1998 = []
for i in range(1,6):  
    d_quantiles_1988.append(df1988['Distance'].quantile(i*0.2))
    d_quantiles_1998.append(df1998['Distance'].quantile(i*0.2))    

In [15]:
def check_quantile_1998(n):
    if  (n<d_quantiles_1998[0]):
        return 'Very short'
    elif (n<d_quantiles_1998[1]):
        return 'Short'
    elif (n<d_quantiles_1998[2]):
        return 'Medium'
    elif (n<d_quantiles_1998[3]):
        return 'Long'
    else:
        return 'Very long'

    
def check_quantile_1988(n):
    if  (n<d_quantiles_1988[0]):
        return 'Very short'
    elif (n<d_quantiles_1988[1]):
        return 'Short'
    elif (n<d_quantiles_1988[2]):
        return 'Medium'
    elif (n<d_quantiles_1988[3]):
        return 'Long'
    else:
        return 'Very long'


"    \ndef check_quantile_1988(n):\n    if  (n<d_quantiles_1988[0]):\n        return 'Very short'\n    elif (n<d_quantiles_1988[1]):\n        return 'Short'\n    elif (n<d_quantiles_1988[2]):\n        return 'Medium'\n    elif (n<d_quantiles_1988[3]):\n        return 'Long'\n    else:\n        return 'Very long'\n"

In [16]:
df1998['DistInterval'] = df1998['Distance'].apply(check_quantile_1998)
df1988['DistInterval'] = df1988['Distance'].apply(check_quantile_1988)

In [17]:
airport_vc_1988 = df1988['Origin'].value_counts()+df1988['Dest'].value_counts()
airport_vc_1998 = df1998['Origin'].value_counts()+df1998['Dest'].value_counts()

s_quantiles_1988 = []
s_quantiles_1998 = []
for i in range(1,6):  
    s_quantiles_1988.append(airport_vc_1988.quantile(i*0.2))
    s_quantiles_1998.append(airport_vc_1998.quantile(i*0.2))    

In [18]:
def check_quantile_1998_s(n):
    if  (n<s_quantiles_1998[0]):
        return 'Very small'
    elif (n<s_quantiles_1998[1]):
        return 'Small'
    elif (n<s_quantiles_1998[2]):
        return 'Medium'
    elif (n<s_quantiles_1998[3]):
        return 'Large'
    else:
        return 'Very large'

    
def check_quantile_1988_s(n):
    if  (n<s_quantiles_1988[0]):
        return 'Very small'
    elif (n<s_quantiles_1988[1]):
        return 'Small'
    elif (n<s_quantiles_1988[2]):
        return 'Medium'
    elif (n<s_quantiles_1988[3]):
        return 'Large'
    else:
        return 'Very large'

def AITA_to_vc_1998(aita):
    return check_quantile_1998_s(airport_vc_1998[aita])

def AITA_to_vc_1988(aita):
    return check_quantile_1988_s(airport_vc_1988[aita])

In [19]:
df1998['OriginSizeInterval'] = df1998['Origin'].apply(AITA_to_vc_1998)


In [20]:
df1988['OriginSizeInterval'] = df1988['Origin'].apply(AITA_to_vc_1988)


In [21]:
df1998['DestSizeInterval'] = df1998['Dest'].apply(AITA_to_vc_1998)


In [22]:
df1988['DestSizeInterval'] = df1988['Dest'].apply(AITA_to_vc_1988)


In [23]:
df1988nc = df1988[df1988['Cancelled']==0].drop(['Cancelled'], axis=1)
df1988c = df1988[df1988['Cancelled']==1].drop(['Cancelled','DepTime','ActualElapsedTime','ArrDelay','DepDelay'], axis=1)
df1998nc = df1998[df1998['Cancelled']==0].drop(['Cancelled'], axis=1)
df1998c = df1998[df1998['Cancelled']==1].drop(['Cancelled','DepTime','ActualElapsedTime','ArrDelay','DepDelay'], axis=1)

In [24]:
df1988nc.to_csv(r'df1988nc.csv',index=False)
df1998nc.to_csv(r'df1998nc.csv',index=False)
df1988c.to_csv(r'df1988c.csv',index=False)
df1998c.to_csv(r'df1998c.csv',index=False)

This setion below is for the corr of arrival and departure delays.

In [25]:
#df1988.sample(n=1000000, random_state=10000).corr(method='pearson')[['ArrDelay','DepDelay']]

In [26]:
#df1988nc.sample(n=1000000, random_state=10000).corr(method='pearson')[['ArrDelay','DepDelay']]

In [27]:
#df1998.sample(n=1000000, random_state=10000).corr(method='pearson')[['ArrDelay','DepDelay']]

In [28]:
#df1998nc.sample(n=1000000, random_state=10000).corr(method='pearson')[['ArrDelay','DepDelay']]

In [29]:
#df1988nc['ArrDelay'].mean()

In [30]:
#df1998nc['ArrDelay'].mean()

In [31]:
#df1988nc.sample(n=1000000, random_state=10000).groupby('DayOfWeek')['ArrDelay','DepDelay'].mean()

In [32]:
#df1998nc.sample(n=1000000, random_state=10000).groupby('DayOfWeek')['ArrDelay','DepDelay'].mean()

In [33]:
#df1998nc.sample(n=100000, random_state=1000000).groupby('UniqueCarrier')['ArrDelay','DepDelay'].mean().sort_values(by='ArrDelay', ascending=False)

In [34]:
#df1988nc.sample(n=100000, random_state=1000000).groupby('UniqueCarrier')['ArrDelay','DepDelay'].mean().sort_values(by='ArrDelay', ascending=False)

This section below is for exploring the dataset - airlines, flight numbers, etc.

In [35]:
#print(len(df1988nc.index))
#print(len(df1988c.index))
#print(df1988nc.columns)
#print(len(df1998nc.index))
#print(len(df1998c.index))
#print(df1998nc.columns)

In [36]:
#df1998nc.sample(n=1000000, random_state=10000).groupby('Month')[['ArrDelay','DepDelay','ActualElapsedTime']].mean()

In [37]:
#df1998nc.sample(n=1000000, random_state=10000).groupby('DayOfWeek')[['ArrDelay','DepDelay','ActualElapsedTime']].mean()

In [38]:
#df1988nc.sample(n=1000000, random_state=10000).groupby('Month')[['ArrDelay','DepDelay','ActualElapsedTime']].mean()

In [39]:
#df1988nc.sample(n=1000000, random_state=10000).groupby('DayOfWeek')[['ArrDelay','DepDelay','ActualElapsedTime']].mean()

In [40]:
#def get_avgs(df_arg, group_by, columns):
#    print(df_arg.sample(n=1000000, random_state=10000).groupby(group_by)[columns].mean())

In [41]:
#get_avgs(df1998nc, 'DayOfWeek', ['ArrDelay','DepDelay','ActualElapsedTime'])

In [42]:
#print(df1988['UniqueCarrier'].value_counts())
#print(df1998['UniqueCarrier'].value_counts())

In [43]:
#print(df1988['Origin'].value_counts())
#print(df1998['Origin'].value_counts())

In [44]:
#print(df1988['Dest'].value_counts())
#print(df1998['Dest'].value_counts())

In [45]:
#df1998['CRSDepTimeInterval'] = df1998['CRSDepTime'].apply(time_to_interval)
#df1998['CRSDepTimeInterval']

In [46]:
#df1998['CRSDepTimeInterval'].value_counts()

In [47]:
#TO DO: Apply the above function to each time parameter, and to both DFs, before preprocessing into the smaller nc/c DFs